<a href="https://colab.research.google.com/github/pedrossntos/saldo-licitacoes/blob/main/saldo_licitacoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, PatternFill, Alignment

In [43]:
def adiciona_linha_excel(linha, arquivo, cabecalho_fornecedor=False):
    try:
        wb = load_workbook(arquivo)
        ws = wb.active
    except:
        wb = Workbook()
        ws = wb.active

    if cabecalho_fornecedor:
        for cell in ws[ws.max_row]:
            cell.font = Font(bold=True if cell.column == 1 else False)

    ws.append(linha)
    wb.save(arquivo)

In [36]:
nome_arquivo = '10.25.1.htm' # ALTERAR O NOME DO ARQUIVO COM O RELATORIO DO EPROCESSOS AQUI
with open(nome_arquivo, encoding='latin-1') as saldo:
  saldo = saldo.read()
soup = BeautifulSoup(saldo, 'html.parser')

In [40]:
divs = soup.find_all('div')

dict_saldo = {
    'fornecedor': [],
    'produto': [],
    'un_medida': [],
    'valor': [],
    'qtd': []
}

# iterar pelas divs para encontrar a primeira com "left: 82px" e conteúdo não vazio
for i, div in enumerate(divs):
    if 'left: 82px' in div.get('style', '') and not div.text.strip():
        fornecedor = divs[i + 3] if i + 3 < len(divs) else None
    elif 'left: 82px' in div.get('style', '') and div.text.strip():
        produto = divs[i + 11].text.strip() if i + 11 < len(divs) else ''
        un_medida = divs[i + 13] if i + 13 < len(divs) else None

        # concatena descrição do produto (se tiver)
        try:
            for j in range(20): # valor aleatorio, loop para quando encontra o proximo produto ou total do fornecedor (div left: 39px)
                if 'left: 77px' in divs[i + j + 11].get('style', ''):
                    produto += ' ' + divs[i + j + 11].text.strip()
                elif 'left: 39px' in divs[i + j + 11].get('style', ''):
                    break
        except:
            pass

        # verifica se é fim de página
        if 'empenho' in produto.strip().lower():
            produto = divs[i + 69].text.strip() if i + 69 < len(divs) else ''
            un_medida = divs[i + 71] if i + 71 < len(divs) else None
            try:
                for j in range(20):
                    if 'left: 77px' in divs[i + j + 69].get('style', ''):
                        produto += ' ' + divs[i + j + 69].text.strip()
                    elif 'left: 39px' in divs[i + j + 69].get('style', ''):
                        break
            except:
                pass

        valor = divs[i + 1] if i + 1 < len(divs) else None
        qtd = divs[i + 2] if i + 2 < len(divs) else None

        dict_saldo['fornecedor'].append(fornecedor.text.strip() if fornecedor else '')
        dict_saldo['produto'].append(produto.strip())
        dict_saldo['un_medida'].append(un_medida.text.strip() if un_medida else '')
        dict_saldo['valor'].append(valor.text.strip() if valor else '')
        dict_saldo['qtd'].append(qtd.text.strip() if qtd else '')

df_saldo = pd.DataFrame(dict_saldo)
df_saldo.head(20)


,fornecedor,produto,un_medida,valor,qtd
0,SONOMAG COMERCIO DE COLCHOES,37528 Colchão Hospitalar com placa de espuma 1...,UN,"190,00","20,00"
1,M. C. COMERCIAL LTDA,66853 Uniforme escolar do tipo calça legging f...,UN,"71,30","30,00"
2,M. C. COMERCIAL LTDA,66854 Uniforme escolar do tipo calça legging f...,UN,"95,00","30,00"
3,M. C. COMERCIAL LTDA,66855 Uniforme escolar do tipo calça legging f...,UN,"72,45","30,00"
4,M. C. COMERCIAL LTDA,66861 Uniforme escolar do tipo calça legging f...,UN,"90,00","35,00"
5,M. C. COMERCIAL LTDA,66860 Uniforme escolar do tipo calça legging f...,UN,"79,35","30,00"
6,M. C. COMERCIAL LTDA,66859 Uniforme escolar do tipo calça legging f...,UN,"79,35","30,00"
7,M. C. COMERCIAL LTDA,"66809 Uniforme escolar do tipo calça, tecido t...",UN,"34,50","35,00"
8,M. C. COMERCIAL LTDA,66879 Uniforme escolar do tipo camiseta de man...,UN,"33,10","30,00"
9,M. C. COMERCIAL LTDA,66880 Uniforme escolar do tipo camiseta de man...,UN,"33,10","30,00"


In [42]:
sucesso = False
while not sucesso:
    try:
        if os.path.exists(nome_arquivo+'.xlsx'):
            print('arquivo ja existe, apagando para nao dar merge com a planilha existente')
            os.remove(nome_arquivo+'.xlsx')
        else:
            fornecedor_ex = ''
            for index, row in df_saldo.iterrows():
                if not row['fornecedor'] == fornecedor_ex:
                    fornecedor_ex = row['fornecedor']
                    adiciona_linha_excel([fornecedor_ex], nome_arquivo+'.xlsx')
                    linha = [row['produto'], row['un_medida'], row['valor'], row['qtd']]
                    adiciona_linha_excel(linha, nome_arquivo+'.xlsx', cabecalho_fornecedor=True)
                else:
                    linha = [row['produto'], row['un_medida'], row['valor'], row['qtd']]
                    adiciona_linha_excel(linha, nome_arquivo+'.xlsx')
            print('planilha criada com sucesso')
            sucesso = True
    except Exception as e:
        print(e)
        print('tentando novamente...')

arquivo ja existe, apagando para nao dar merge com a planilha existente
planilha criada com sucesso
